In [ ]:
# fetching data from csv file

import pandas as pd

In [ ]:
matches = pd.read_csv("matches.csv")


In [ ]:
matches.head()

In [ ]:
matches.shape

In [ ]:
matches["team"].value_counts()


In [ ]:

matches[matches["team"] == "Liverpool"]

In [ ]:
matches["round"].value_counts()

In [ ]:
matches.dtypes

In [ ]:
# ml only works with numerical data or float
matches['date'] = pd.to_datetime(matches['date']) #convert date to datetime format
matches.dtypes

In [ ]:
# predictors 
matches['venue_code'] = matches['venue'].astype('category').cat.codes#from sting to caterogy to numbers

In [ ]:
matches["opp_codes"] = matches['opponent'].astype('category').cat.codes


In [ ]:
matches

In [ ]:
matches["hour"] = matches['time'].str.replace(":.+", "", regex=True).astype("int")

In [ ]:
matches


In [ ]:
matches["day_code"] = matches['date'].dt.dayofweek

In [ ]:
matches


In [ ]:
matches["target"] = (matches['result'] == "W").astype("int")

In [ ]:
matches

In [ ]:
# creating model
from sklearn.ensemble import RandomForestClassifier #Random Forest model can pickup non linear relationships


In [ ]:
# random forest model is a series of decision trees the more the estimator the better the model generalizes the more accurate but the longer it takes to train
# min samples is the minimum number of samples required to split an internal node the higher the less overfitting but the less accurate
#  random state is for reproducibility 
rf = RandomForestClassifier(n_estimators=50, min_samples_split=10, random_state=1)

In [ ]:
train = matches[matches['date'] < '2022-01-01']
test = matches[matches['date'] >= '2022-01-01']

In [ ]:
predictors = ['venue_code', 'opp_codes', 'hour', 'day_code']

In [ ]:
rf.fit(train[predictors], train['target'])

In [ ]:
preds = rf.predict(test[predictors])

In [ ]:
from sklearn.metrics import accuracy_score, precision_score

In [ ]:
acc = accuracy_score(test['target'], preds)

In [ ]:
acc

In [ ]:
combined = pd.DataFrame(dict(actual=test['target'], prediction=preds))

In [ ]:
pd.crosstab(index=combined["actual"], columns=combined["prediction"])

In [ ]:
precision_score(test['target'], preds)


In [ ]:
# creates a dataframe for each team showing their matches   
grouped_matches = matches.groupby("team")

In [ ]:
group = grouped_matches.get_group("Liverpool")

In [ ]:
def rolling_averages(group, col, new_cols):
    group = group.sort_values("date")
    rolling_stats = group[col].rolling(3, closed='left')
    group[new_cols] = rolling_stats
    group = group.dropna(subset=new_cols)
    

In [ ]:
cols = ['gf', 'ga', 'sh', 'sot', 'dist', 'fk', 'pk', 'pkatt']
new_cols = [f'{c}_rolling' for c in cols]



In [ ]:
new_cols

In [ ]:
rolling_averages(group, cols, new_cols)

In [ ]:
matches_rolling = matches.groupby("team").apply(lambda x: rolling_averages(x, cols, new_cols))

In [ ]:
matches_rolling

In [ ]:
matches_rolling = matches_rolling.droplevel("team")

In [ ]:
matches_rolling

In [ ]:
matches_rolling.index = range(matches_rolling.shape[0])

In [ ]:
matches_rolling

In [ ]:
# retraining the model for rolling averages features
def make_predictions(data, predictors, model=rf):
    train = data[data['date'] < '2022-01-01']
    test = data[data['date'] >= '2022-01-01']

    rf.fit(train[predictors], train['target'])

    preds = rf.predict(test[predictors])
    combined = pd.DataFrame(dict(actual=test['target'], prediction=preds), index=test.index)
    precision = precision_score(test['target'], preds)
    return combined, precision

In [ ]:
combined, precision = make_predictions(matches_rolling, predictors + new_cols)  


In [ ]:
precision

In [ ]:
combined

In [ ]:
combined = combined.merge(matches_rolling[["team", "opponent", "date", "result"]], left_index=True, right_index=True)

In [ ]:
combined

In [ ]:
# combining home and away predictions
class MissingDict(dict):
    __missing__ = lambda self, key: key

maps_values = {
    "Brighton and Hove Albion": "Brighton",
    "Manchester United": "Manchester Utd",
    "Newcastle United": "Newcaste Utd",
    "Tottenham Hotspur": "Tottenham",
    "West Ham United": "West Ham",
    "Wolverhampton Wanderers": "Wolves"
}

mapping = MissingDict(**maps_values)

In [ ]:
mapping["Arsenal"]


In [ ]:
mapping["West Ham United"]

In [ ]:
combined["new_teams"] = combined["team"].map(mapping)